<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE3-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Complete Cached MLP with Incremental Training
# Grid search once → Train once → Reuse everything → Incremental training option

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import hashlib
import json
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.regularizers import l1_l2
import warnings
warnings.filterwarnings('ignore')

class CompleteCachedMLP:
    """
    Complete MLP solution with full caching and incremental training
    - Grid search: Done once, cached forever
    - Model training: Done once, cached forever
    - Results: Saved for reuse
    - Incremental training: Available on demand
    """

    def __init__(self, output_dir='/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/'):
        self.output_dir = output_dir
        self.y_scaler = StandardScaler()
        self.model = None
        self.best_params = None
        self.correlation_analysis = None

        # Organized directory structure
        self.cache_dir = f"{output_dir}mlp_cache/"
        self.models_dir = f"{output_dir}mlp_models/"
        self.results_dir = f"{output_dir}mlp_results/"
        self.checkpoints_dir = f"{output_dir}mlp_checkpoints/"
        self.incremental_dir = f"{output_dir}mlp_incremental/"

        for directory in [self.cache_dir, self.models_dir, self.results_dir,
                         self.checkpoints_dir, self.incremental_dir]:
            os.makedirs(directory, exist_ok=True)

        print(f"📁 Directories initialized:")
        print(f"   Cache: {self.cache_dir}")
        print(f"   Models: {self.models_dir}")
        print(f"   Results: {self.results_dir}")

    def get_data_hash(self, data_filename, windows_filename):
        """Generate unique hash for data files"""
        data_path = os.path.join(self.output_dir, data_filename)
        windows_path = os.path.join(self.output_dir, windows_filename)

        hash_input = f"{data_filename}_{windows_filename}"

        if os.path.exists(data_path) and os.path.exists(windows_path):
            data_stat = os.stat(data_path)
            windows_stat = os.stat(windows_path)
            hash_input += f"_{data_stat.st_size}_{data_stat.st_mtime}_{windows_stat.st_size}_{windows_stat.st_mtime}"

        return hashlib.md5(hash_input.encode()).hexdigest()[:12]

    def load_vae_files(self, data_filename, windows_filename):
        """Load VAE output files"""
        print("📊 Loading VAE output files...")

        data_path = os.path.join(self.output_dir, data_filename)
        windows_path = os.path.join(self.output_dir, windows_filename)

        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Data file not found: {data_path}")
        if not os.path.exists(windows_path):
            raise FileNotFoundError(f"Windows file not found: {windows_path}")

        x = np.load(data_path)
        y = np.load(windows_path)

        print(f"✅ Loaded: X={x.shape}, y={y.shape}")
        print(f"   Data range: [{np.min(x):.4f}, {np.max(x):.4f}]")
        print(f"   Windows range: [{np.min(y):.0f}, {np.max(y):.0f}]")

        if x.shape[0] != y.shape[0]:
            raise ValueError(f"Sample mismatch: data={x.shape[0]}, windows={y.shape[0]}")

        return x, y

    def analyze_correlations(self, x, y):
        """Analyze correlations and recommend architecture"""
        print("\n🔗 Analyzing correlations...")

        correlations = []
        for i in range(x.shape[1]):
            if np.std(x[:, i]) > 1e-8:
                corr = np.corrcoef(x[:, i], y)[0, 1]
                if not np.isnan(corr) and not np.isinf(corr):
                    correlations.append(abs(corr))

        max_corr = max(correlations) if correlations else 0
        mean_corr = np.mean(correlations) if correlations else 0

        # Architecture recommendation
        if max_corr < 0.01:
            arch_recommendation = "VERY_DEEP_WIDE"
        elif max_corr < 0.02:
            arch_recommendation = "DEEP_WIDE"
        elif max_corr < 0.05:
            arch_recommendation = "DEEP"
        elif max_corr < 0.1:
            arch_recommendation = "WIDE"
        else:
            arch_recommendation = "STANDARD"

        analysis = {
            'max_correlation': max_corr,
            'mean_correlation': mean_corr,
            'total_features': len(correlations),
            'architecture_recommendation': arch_recommendation
        }

        print(f"   Max correlation: {max_corr:.6f}")
        print(f"   Mean correlation: {mean_corr:.6f}")
        print(f"   Architecture: {arch_recommendation}")

        return analysis

    def define_architecture(self, input_dim, architecture_type):
        """Define architecture based on type"""
        configs = {
            "VERY_DEEP_WIDE": {
                'layers': [
                    {'units': 2048, 'dropout': 0.4, 'batch_norm': True},
                    {'units': 1024, 'dropout': 0.4, 'batch_norm': True},
                    {'units': 512, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 256, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 128, 'dropout': 0.2, 'batch_norm': True},
                    {'units': 64, 'dropout': 0.2, 'batch_norm': False},
                    {'units': 32, 'dropout': 0.1, 'batch_norm': False}
                ],
                'default_lr': 0.001,
                'default_batch_size': 32
            },
            "DEEP_WIDE": {
                'layers': [
                    {'units': 1024, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 512, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 256, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 128, 'dropout': 0.2, 'batch_norm': True},
                    {'units': 64, 'dropout': 0.2, 'batch_norm': False},
                    {'units': 32, 'dropout': 0.1, 'batch_norm': False}
                ],
                'default_lr': 0.002,
                'default_batch_size': 64
            },
            "DEEP": {
                'layers': [
                    {'units': 512, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 256, 'dropout': 0.3, 'batch_norm': True},
                    {'units': 128, 'dropout': 0.2, 'batch_norm': True},
                    {'units': 64, 'dropout': 0.2, 'batch_norm': False},
                    {'units': 32, 'dropout': 0.1, 'batch_norm': False}
                ],
                'default_lr': 0.002,
                'default_batch_size': 64
            },
            "WIDE": {
                'layers': [
                    {'units': 512, 'dropout': 0.2, 'batch_norm': True},
                    {'units': 256, 'dropout': 0.2, 'batch_norm': True},
                    {'units': 128, 'dropout': 0.1, 'batch_norm': False},
                    {'units': 64, 'dropout': 0.1, 'batch_norm': False}
                ],
                'default_lr': 0.001,
                'default_batch_size': 128
            },
            "STANDARD": {
                'layers': [
                    {'units': 256, 'dropout': 0.2, 'batch_norm': True},
                    {'units': 128, 'dropout': 0.2, 'batch_norm': False},
                    {'units': 64, 'dropout': 0.1, 'batch_norm': False},
                    {'units': 32, 'dropout': 0.1, 'batch_norm': False}
                ],
                'default_lr': 0.001,
                'default_batch_size': 128
            }
        }

        config = configs[architecture_type]
        return config['layers'], config['default_lr'], config['default_batch_size']

    def build_mlp_model(self, input_dim, layers, learning_rate=0.001, l1_reg=0.001, l2_reg=0.01):
        """Build MLP model"""
        model = Sequential()

        # First layer
        first_layer = layers[0]
        model.add(Dense(
            first_layer['units'],
            input_dim=input_dim,
            activation='relu',
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)
        ))

        if first_layer['batch_norm']:
            model.add(BatchNormalization())
        if first_layer['dropout'] > 0:
            model.add(Dropout(first_layer['dropout']))

        # Hidden layers
        for layer in layers[1:]:
            model.add(Dense(
                layer['units'],
                activation='relu',
                kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)
            ))

            if layer['batch_norm']:
                model.add(BatchNormalization())
            if layer['dropout'] > 0:
                model.add(Dropout(layer['dropout']))

        # Output layer
        model.add(Dense(1, activation='linear'))

        # Compile
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
        model.compile(loss='huber', optimizer=optimizer, metrics=['mae', 'mse'])

        return model

    # ========== CACHING SYSTEM ==========

    def save_grid_search_cache(self, best_params, architecture_type, correlation_analysis, data_hash):
        """Save grid search results"""
        cache_file = f"{self.cache_dir}grid_search_{data_hash}.json"

        cache_data = {
            'data_hash': data_hash,
            'architecture_type': architecture_type,
            'best_params': best_params,
            'correlation_analysis': correlation_analysis,
            'timestamp': pd.Timestamp.now().isoformat(),
            'completed': True
        }

        with open(cache_file, 'w') as f:
            json.dump(cache_data, f, indent=2)

        print(f"💾 Grid search cached: {cache_file}")

    def load_grid_search_cache(self, data_hash):
        """Load grid search results"""
        cache_file = f"{self.cache_dir}grid_search_{data_hash}.json"

        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'r') as f:
                    cache_data = json.load(f)

                print(f"✅ Grid search cache found:")
                print(f"   Architecture: {cache_data['architecture_type']}")
                print(f"   Max correlation: {cache_data['correlation_analysis']['max_correlation']:.6f}")
                print(f"   Cached: {cache_data['timestamp']}")

                return (cache_data['best_params'],
                       cache_data['architecture_type'],
                       cache_data['correlation_analysis'])
            except Exception as e:
                print(f"⚠️ Grid search cache corrupted: {e}")

        return None, None, None

    def save_trained_model_cache(self, model, y_scaler, metadata, data_hash):
        """Save trained model"""
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")

        model_file = f"{self.models_dir}model_{data_hash}_{timestamp}.keras"
        scaler_file = f"{self.models_dir}scaler_{data_hash}_{timestamp}.pkl"
        metadata_file = f"{self.models_dir}metadata_{data_hash}_{timestamp}.json"

        # Save model
        model.save(model_file)

        # Save scaler
        with open(scaler_file, 'wb') as f:
            pickle.dump(y_scaler, f)

        # Save metadata
        full_metadata = {
            'data_hash': data_hash,
            'model_file': model_file,
            'scaler_file': scaler_file,
            'timestamp': timestamp,
            **metadata
        }

        with open(metadata_file, 'w') as f:
            json.dump(full_metadata, f, indent=2)

        # Update latest pointer
        latest_file = f"{self.models_dir}latest_{data_hash}.json"
        latest_data = {
            'model_file': model_file,
            'scaler_file': scaler_file,
            'metadata_file': metadata_file,
            'timestamp': timestamp
        }

        with open(latest_file, 'w') as f:
            json.dump(latest_data, f, indent=2)

        print(f"💾 Model saved: {model_file}")
        return model_file, scaler_file, metadata_file

    def load_trained_model_cache(self, data_hash):
        """Load trained model"""
        latest_file = f"{self.models_dir}latest_{data_hash}.json"

        if os.path.exists(latest_file):
            try:
                with open(latest_file, 'r') as f:
                    latest_data = json.load(f)

                model_file = latest_data['model_file']
                scaler_file = latest_data['scaler_file']
                metadata_file = latest_data['metadata_file']

                if all(os.path.exists(f) for f in [model_file, scaler_file, metadata_file]):
                    # Load model
                    model = keras.models.load_model(model_file)

                    # Load scaler
                    with open(scaler_file, 'rb') as f:
                        y_scaler = pickle.load(f)

                    # Load metadata
                    with open(metadata_file, 'r') as f:
                        metadata = json.load(f)

                    print(f"✅ Trained model found:")
                    print(f"   Trained: {latest_data['timestamp']}")
                    print(f"   R²: {metadata.get('final_r2', 'Unknown')}")

                    return model, y_scaler, metadata
            except Exception as e:
                print(f"⚠️ Model cache corrupted: {e}")

        return None, None, None

    def save_test_results(self, x_test, y_true, y_pred, evaluation_results, data_hash):
        """Save test data and results"""
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")

        # Save test data
        test_file = f"{self.results_dir}test_data_{data_hash}_{timestamp}.npz"
        np.savez_compressed(
            test_file,
            x_test=x_test,
            y_true=y_true,
            y_pred=y_pred
        )

        # Save results
        results_file = f"{self.results_dir}results_{data_hash}_{timestamp}.json"

        # Convert numpy types for JSON
        json_results = {}
        for key, value in evaluation_results.items():
            if isinstance(value, np.ndarray):
                json_results[key] = value.tolist()
            elif isinstance(value, (np.integer, np.floating)):
                json_results[key] = float(value)
            else:
                json_results[key] = value

        json_results['timestamp'] = timestamp

        with open(results_file, 'w') as f:
            json.dump(json_results, f, indent=2)

        # Update latest pointer
        latest_file = f"{self.results_dir}latest_{data_hash}.json"
        latest_data = {
            'test_file': test_file,
            'results_file': results_file,
            'timestamp': timestamp
        }

        with open(latest_file, 'w') as f:
            json.dump(latest_data, f, indent=2)

        print(f"💾 Results saved:")
        print(f"   Test data: {test_file}")
        print(f"   Results: {results_file}")

        return test_file, results_file

    def load_test_results(self, data_hash):
        """Load test results"""
        latest_file = f"{self.results_dir}latest_{data_hash}.json"

        if os.path.exists(latest_file):
            try:
                with open(latest_file, 'r') as f:
                    latest_data = json.load(f)

                test_file = latest_data['test_file']
                results_file = latest_data['results_file']

                if os.path.exists(test_file) and os.path.exists(results_file):
                    # Load test data
                    test_data = np.load(test_file)

                    # Load results
                    with open(results_file, 'r') as f:
                        results = json.load(f)

                    print(f"✅ Test results found:")
                    print(f"   Generated: {latest_data['timestamp']}")
                    print(f"   R²: {results.get('r2', 'Unknown')}")

                    return test_data, results
            except Exception as e:
                print(f"⚠️ Results cache corrupted: {e}")

        return None, None

    # ========== TRAINING SYSTEM ==========

    def grid_search_hyperparameters(self, x_train, y_train, x_val, y_val, architecture_type, base_lr, base_batch_size):
        """Grid search with progress tracking"""
        print(f"\n🔍 Grid search for {architecture_type}...")

        # Define search space
        if architecture_type in ["VERY_DEEP_WIDE", "DEEP_WIDE"]:
            lr_options = [base_lr * 0.5, base_lr, base_lr * 2]
            batch_options = [base_batch_size // 2, base_batch_size, base_batch_size * 2]
            l1_options = [0.0001, 0.001, 0.01]
            l2_options = [0.001, 0.01, 0.1]
        else:
            lr_options = [base_lr * 0.5, base_lr, base_lr * 1.5]
            batch_options = [base_batch_size, base_batch_size * 2]
            l1_options = [0.0001, 0.001]
            l2_options = [0.001, 0.01]

        total = len(lr_options) * len(batch_options) * len(l1_options) * len(l2_options)
        print(f"   Testing {total} combinations...")

        best_score = -np.inf
        best_params = None
        count = 0

        layers, _, _ = self.define_architecture(x_train.shape[1], architecture_type)

        for lr in lr_options:
            for batch in batch_options:
                for l1 in l1_options:
                    for l2 in l2_options:
                        count += 1
                        try:
                            print(f"   {count}/{total}: LR={lr:.4f}, Batch={batch}, L1={l1:.4f}, L2={l2:.4f}")

                            model = self.build_mlp_model(x_train.shape[1], layers, lr, l1, l2)

                            history = model.fit(
                                x_train, y_train,
                                validation_data=(x_val, y_val),
                                epochs=100,
                                batch_size=int(batch),
                                callbacks=[EarlyStopping(monitor='val_loss', patience=20)],
                                verbose=0
                            )

                            score = -min(history.history['val_loss'])

                            if score > best_score:
                                best_score = score
                                best_params = {
                                    'learning_rate': lr,
                                    'batch_size': int(batch),
                                    'l1_reg': l1,
                                    'l2_reg': l2
                                }
                                print(f"     ✓ New best: {-score:.6f}")

                            del model
                            keras.backend.clear_session()

                        except Exception as e:
                            print(f"     ✗ Failed: {e}")

        if best_params:
            print(f"\n🏆 Best parameters:")
            for k, v in best_params.items():
                print(f"   {k}: {v}")
        else:
            best_params = {
                'learning_rate': base_lr,
                'batch_size': base_batch_size,
                'l1_reg': 0.001,
                'l2_reg': 0.01
            }

        return best_params

    def train_final_model(self, x_train, y_train, x_val, y_val, architecture_type, best_params, data_hash):
        """Train final model with checkpointing"""
        print(f"\n🚀 Training final {architecture_type} model...")

        # Build model
        layers, _, _ = self.define_architecture(x_train.shape[1], architecture_type)
        self.model = self.build_mlp_model(
            x_train.shape[1], layers,
            best_params['learning_rate'],
            best_params['l1_reg'],
            best_params['l2_reg']
        )

        print(f"   Parameters: {self.model.count_params():,}")

        # Checkpoint setup
        checkpoint_file = f"{self.checkpoints_dir}training_{data_hash}.weights.h5"

        # Callbacks
        callbacks = [
            ModelCheckpoint(
                checkpoint_file,
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True,
                verbose=1
            ),
            EarlyStopping(
                monitor='val_loss',
                patience=100,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=30,
                min_lr=1e-7,
                verbose=1
            )
        ]

        # Train
        history = self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=1000,
            batch_size=best_params['batch_size'],
            callbacks=callbacks,
            verbose=1
        )

        print("✅ Training complete!")
        return history

    def evaluate_performance(self, x_test, y_test, history):
        """Evaluate model performance"""
        print("\n📊 Evaluating performance...")

        # Predictions
        y_pred_scaled = self.model.predict(x_test, verbose=0)
        y_pred = self.y_scaler.inverse_transform(y_pred_scaled).flatten()
        y_true = self.y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Metrics
        r2 = r2_score(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)

        # Accuracy within tolerance
        acc_05 = np.mean(np.abs(y_true - y_pred) <= 0.5) * 100
        acc_1 = np.mean(np.abs(y_true - y_pred) <= 1) * 100
        acc_15 = np.mean(np.abs(y_true - y_pred) <= 1.5) * 100
        acc_2 = np.mean(np.abs(y_true - y_pred) <= 2) * 100

        results = {
            'r2': r2, 'mse': mse, 'mae': mae, 'rmse': rmse,
            'acc_05': acc_05, 'acc_1': acc_1, 'acc_15': acc_15, 'acc_2': acc_2,
            'y_true': y_true, 'y_pred': y_pred
        }

        print(f"📈 Results:")
        print(f"   R²: {r2:.6f}")
        print(f"   MAE: {mae:.4f}")
        print(f"   Accuracy ±1: {acc_1:.1f}%")

        # Plot results
        self.plot_results(y_true, y_pred, history, r2, mae)

        return results

    def plot_results(self, y_true, y_pred, history, r2, mae):
        """Plot comprehensive results"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        # Training history
        axes[0,0].plot(history.history['loss'], label='Training')
        axes[0,0].plot(history.history['val_loss'], label='Validation')
        axes[0,0].set_title('Loss')
        axes[0,0].set_yscale('log')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)

        # MAE history
        axes[0,1].plot(history.history['mae'], label='Training')
        axes[0,1].plot(history.history['val_mae'], label='Validation')
        axes[0,1].set_title('MAE')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)

        # Predictions scatter
        axes[0,2].scatter(y_true, y_pred, alpha=0.6, s=3)
        axes[0,2].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
        axes[0,2].set_xlabel('True')
        axes[0,2].set_ylabel('Predicted')
        axes[0,2].set_title(f'Predictions (R²={r2:.4f})')
        axes[0,2].grid(True, alpha=0.3)

        # Residuals
        residuals = y_true - y_pred
        axes[1,0].scatter(y_pred, residuals, alpha=0.6, s=3)
        axes[1,0].axhline(y=0, color='r', linestyle='--')
        axes[1,0].set_xlabel('Predicted')
        axes[1,0].set_ylabel('Residuals')
        axes[1,0].set_title('Residuals')
        axes[1,0].grid(True, alpha=0.3)

        # Error distribution
        axes[1,1].hist(residuals, bins=50, alpha=0.7)
        axes[1,1].set_xlabel('Residuals')
        axes[1,1].set_ylabel('Frequency')
        axes[1,1].set_title('Error Distribution')
        axes[1,1].grid(True, alpha=0.3)

        # Accuracy bars
        tolerances = [0.5, 1.0, 1.5, 2.0]
        accuracies = [np.mean(np.abs(residuals) <= tol) * 100 for tol in tolerances]
        axes[1,2].bar(tolerances, accuracies, alpha=0.7)
        axes[1,2].set_xlabel('Tolerance')
        axes[1,2].set_ylabel('Accuracy (%)')
        axes[1,2].set_title('Accuracy vs Tolerance')
        axes[1,2].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    # ========== INCREMENTAL TRAINING ==========

    def incremental_training(self, x_new, y_new, data_hash, epochs=100, learning_rate_factor=0.1):
        """Incremental training on new data"""
        if self.model is None:
            raise ValueError("No base model found! Train initial model first.")

        print(f"\n🔄 Incremental training on {len(x_new)} new samples...")

        # Scale new targets
        y_new_scaled = self.y_scaler.transform(y_new.reshape(-1, 1)).flatten()

        # Reduce learning rate
        original_lr = self.model.optimizer.learning_rate.numpy()
        new_lr = original_lr * learning_rate_factor
        self.model.optimizer.learning_rate.assign(new_lr)

        print(f"   Learning rate: {original_lr:.6f} → {new_lr:.6f}")

        # Split new data
        x_train_new, x_val_new, y_train_new, y_val_new = train_test_split(
            x_new, y_new_scaled, test_size=0.2, random_state=42
        )

        # Incremental checkpoint
        inc_checkpoint = f"{self.incremental_dir}incremental_{data_hash}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.weights.h5"

        # Callbacks for incremental training
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
            ModelCheckpoint(inc_checkpoint, monitor='val_loss', save_best_only=True, save_weights_only=True)
        ]

        # Train incrementally
        history = self.model.fit(
            x_train_new, y_train_new,
            validation_data=(x_val_new, y_val_new),
            epochs=epochs,
            batch_size=64,
            callbacks=callbacks,
            verbose=1
        )

        # Save incremental model
        inc_model_file = f"{self.incremental_dir}incremental_model_{data_hash}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.keras"
        self.model.save(inc_model_file)

        print(f"✅ Incremental training complete!")
        print(f"💾 Incremental model saved: {inc_model_file}")

        return history, inc_model_file

    # ========== MAIN INTERFACE ==========

    def check_status(self, data_filename, windows_filename):
        """Check what's already completed"""
        data_hash = self.get_data_hash(data_filename, windows_filename)

        print(f"\n📋 STATUS CHECK (Hash: {data_hash})")
        print("="*50)

        # Check grid search
        grid_params, arch_type, corr_analysis = self.load_grid_search_cache(data_hash)
        grid_done = grid_params is not None

        # Check trained model
        model, scaler, metadata = self.load_trained_model_cache(data_hash)
        training_done = model is not None

        # Check results
        test_data, results = self.load_test_results(data_hash)
        results_done = test_data is not None

        print(f"Grid Search: {'✅ DONE' if grid_done else '❌ NEEDED'}")
        print(f"Model Training: {'✅ DONE' if training_done else '❌ NEEDED'}")
        print(f"Test Results: {'✅ DONE' if results_done else '❌ NEEDED'}")
        print(f"Complete Pipeline: {'✅ READY' if all([grid_done, training_done, results_done]) else '❌ INCOMPLETE'}")

        if grid_done:
            print(f"   Architecture: {arch_type}")
            print(f"   Max correlation: {corr_analysis['max_correlation']:.6f}")

        if training_done:
            print(f"   Final R²: {metadata.get('final_r2', 'Unknown')}")

        print("="*50)

        return {
            'data_hash': data_hash,
            'grid_done': grid_done,
            'training_done': training_done,
            'results_done': results_done,
            'all_done': all([grid_done, training_done, results_done])
        }

    def run_complete_pipeline(self, data_filename, windows_filename, force_retrain=False):
        """
        Complete pipeline with full caching

        Args:
            data_filename: VAE synthetic data file
            windows_filename: VAE windows file
            force_retrain: Force complete retrain ignoring cache
        """
        print("="*80)
        print("🎯 COMPLETE CACHED MLP PIPELINE")
        print("="*80)

        data_hash = self.get_data_hash(data_filename, windows_filename)
        print(f"🔑 Data hash: {data_hash}")

        # Check status
        status = self.check_status(data_filename, windows_filename)

        if status['all_done'] and not force_retrain:
            print("\n🎉 EVERYTHING ALREADY COMPLETED!")
            print("Loading from cache...")

            # Load everything
            grid_params, arch_type, corr_analysis = self.load_grid_search_cache(data_hash)
            model, y_scaler, metadata = self.load_trained_model_cache(data_hash)
            test_data, results = self.load_test_results(data_hash)

            # Set instance variables
            self.model = model
            self.y_scaler = y_scaler
            self.best_params = grid_params
            self.correlation_analysis = corr_analysis

            print(f"\n📊 CACHED RESULTS:")
            print(f"   Architecture: {arch_type}")
            print(f"   R²: {results['r2']:.6f}")
            print(f"   MAE: {results['mae']:.4f}")
            print(f"   Accuracy ±1: {results['acc_1']:.1f}%")

            return {
                'status': 'loaded_from_cache',
                'model': model,
                'results': results,
                'test_data': test_data,
                'metadata': metadata
            }

        if force_retrain:
            print("\n🔄 FORCED RETRAIN - Ignoring all caches")

        try:
            # 1. Load data
            x, y = self.load_vae_files(data_filename, windows_filename)

            # 2. Grid search (with caching)
            if not force_retrain and status['grid_done']:
                print("\n⚡ Using cached grid search")
                best_params, architecture_type, correlation_analysis = self.load_grid_search_cache(data_hash)
            else:
                print("\n🔍 Running grid search...")
                # Analyze correlations
                correlation_analysis = self.analyze_correlations(x, y)
                architecture_type = correlation_analysis['architecture_recommendation']

                # Split data
                x_temp, x_test, y_temp, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
                x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

                # Scale for grid search
                temp_scaler = StandardScaler()
                y_train_scaled = temp_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
                y_val_scaled = temp_scaler.transform(y_val.reshape(-1, 1)).flatten()

                # Get architecture config
                layers, default_lr, default_batch_size = self.define_architecture(x_train.shape[1], architecture_type)

                # Run grid search
                best_params = self.grid_search_hyperparameters(
                    x_train, y_train_scaled, x_val, y_val_scaled,
                    architecture_type, default_lr, default_batch_size
                )

                # Cache grid search results
                self.save_grid_search_cache(best_params, architecture_type, correlation_analysis, data_hash)

            # Set instance variables
            self.correlation_analysis = correlation_analysis
            self.best_params = best_params

            # 3. Model training (with caching)
            if not force_retrain and status['training_done']:
                print("\n⚡ Using cached trained model")
                model, y_scaler, metadata = self.load_trained_model_cache(data_hash)
                self.model = model
                self.y_scaler = y_scaler

                # Check if results also cached
                test_data, results = self.load_test_results(data_hash)
                if test_data is not None:
                    print("✅ Test results also cached")
                    return {
                        'status': 'loaded_model_from_cache',
                        'model': model,
                        'results': results,
                        'test_data': test_data,
                        'metadata': metadata
                    }
            else:
                print("\n🚀 Training model...")

                # Prepare data (if not already done)
                if 'x_train' not in locals():
                    x_temp, x_test, y_temp, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
                    x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

                # Scale targets
                y_train_scaled = self.y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
                y_val_scaled = self.y_scaler.transform(y_val.reshape(-1, 1)).flatten()
                y_test_scaled = self.y_scaler.transform(y_test.reshape(-1, 1)).flatten()

                print(f"   Data split: Train={len(x_train)}, Val={len(x_val)}, Test={len(x_test)}")

                # Train model
                history = self.train_final_model(
                    x_train, y_train_scaled, x_val, y_val_scaled,
                    architecture_type, best_params, data_hash
                )

                # Evaluate
                evaluation_results = self.evaluate_performance(x_test, y_test_scaled, history)

                # Save trained model
                training_metadata = {
                    'architecture_type': architecture_type,
                    'best_params': best_params,
                    'correlation_analysis': correlation_analysis,
                    'final_r2': evaluation_results['r2'],
                    'final_mae': evaluation_results['mae'],
                    'training_samples': len(x_train),
                    'test_samples': len(x_test)
                }

                self.save_trained_model_cache(self.model, self.y_scaler, training_metadata, data_hash)

                # Save test results
                self.save_test_results(
                    x_test, evaluation_results['y_true'], evaluation_results['y_pred'],
                    evaluation_results, data_hash
                )

            # Final summary
            print("\n" + "="*80)
            print("🎉 PIPELINE COMPLETE!")
            print("="*80)
            print(f"Architecture: {architecture_type}")
            print(f"Max Correlation: {correlation_analysis['max_correlation']:.6f}")
            print(f"Final R²: {evaluation_results['r2']:.6f}")
            print(f"Final MAE: {evaluation_results['mae']:.4f}")
            print(f"Accuracy ±1: {evaluation_results['acc_1']:.1f}%")
            print(f"\n💾 All results cached for future use!")
            print("="*80)

            return {
                'status': 'newly_completed',
                'model': self.model,
                'results': evaluation_results,
                'correlation_analysis': correlation_analysis,
                'best_params': best_params,
                'architecture_type': architecture_type
            }

        except Exception as e:
            print(f"❌ Pipeline failed: {e}")
            import traceback
            traceback.print_exc()
            return None

# ========== SIMPLE INTERFACE FUNCTIONS ==========

def run_mlp_pipeline(data_filename, windows_filename, force_retrain=False):
    """
    Simple interface for complete MLP pipeline

    Args:
        data_filename: e.g., 'generated-data-OPTIMIZED.npy'
        windows_filename: e.g., 'generated-data-true-window-OPTIMIZED.npy'
        force_retrain: Force complete retrain ignoring all caches
    """
    mlp = CompleteCachedMLP()
    return mlp.run_complete_pipeline(data_filename, windows_filename, force_retrain)

def check_mlp_status(data_filename, windows_filename):
    """Check what's already completed for given data files"""
    mlp = CompleteCachedMLP()
    return mlp.check_status(data_filename, windows_filename)

def run_incremental_training(data_filename, windows_filename, x_new, y_new, epochs=100):
    """
    Run incremental training on new data

    Args:
        data_filename: Original data file (for loading base model)
        windows_filename: Original windows file
        x_new: New feature data
        y_new: New target data
        epochs: Training epochs for incremental learning
    """
    mlp = CompleteCachedMLP()

    # Load base model
    data_hash = mlp.get_data_hash(data_filename, windows_filename)
    model, y_scaler, metadata = mlp.load_trained_model_cache(data_hash)

    if model is None:
        raise ValueError("No trained base model found! Run main pipeline first.")

    mlp.model = model
    mlp.y_scaler = y_scaler

    # Run incremental training
    return mlp.incremental_training(x_new, y_new, data_hash, epochs)

def get_test_predictions(data_filename, windows_filename):
    """
    Get saved test predictions for streaming simulation

    Returns:
        dict with 'x_test', 'y_true', 'y_pred' arrays
    """
    mlp = CompleteCachedMLP()
    data_hash = mlp.get_data_hash(data_filename, windows_filename)

    test_data, results = mlp.load_test_results(data_hash)
    if test_data is None:
        raise ValueError("No test results found! Run main pipeline first.")

    return {
        'x_test': test_data['x_test'],
        'y_true': test_data['y_true'],
        'y_pred': test_data['y_pred'],
        'results': results
    }

# ========== MAIN EXECUTION ==========

if __name__ == "__main__":
    print("🎯 Complete Cached MLP System")

    # SPECIFY YOUR FILES
    data_file = 'generated-data-OPTIMIZED.npy'
    windows_file = 'generated-data-true-window-OPTIMIZED.npy'

    # Check status first
    print("Checking current status...")
    status = check_mlp_status(data_file, windows_file)

    if status['all_done']:
        print("Everything already completed! Loading results...")
        results = run_mlp_pipeline(data_file, windows_file)
        print(f"Final R²: {results['results']['r2']:.6f}")
    else:
        print("Running pipeline...")
        results = run_mlp_pipeline(data_file, windows_file)

        if results:
            print(f"Pipeline complete! R²: {results['results']['r2']:.6f}")
        else:
            print("Pipeline failed!")

# ========== USAGE EXAMPLES ==========

"""
# Basic usage - runs once, caches everything
results = run_mlp_pipeline('data.npy', 'windows.npy')

# Check what's already done
status = check_mlp_status('data.npy', 'windows.npy')

# Force retrain everything
results = run_mlp_pipeline('data.npy', 'windows.npy', force_retrain=True)

# Get test predictions for next steps
test_data = get_test_predictions('data.npy', 'windows.npy')
x_test = test_data['x_test']
y_true = test_data['y_true']
y_pred = test_data['y_pred']

# Incremental training with new data
new_x = np.random.randn(1000, 50)  # New synthetic data
new_y = np.random.randint(5, 25, 1000)  # New windows
inc_history, inc_model = run_incremental_training(
    'data.npy', 'windows.npy', new_x, new_y, epochs=50
)
"""

🎯 Adaptive MLP Analysis
Analyzing files: generated-data-OPTIMIZED.npy, generated-data-true-window-OPTIMIZED.npy
📁 Output directory: /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/
🎯 ADAPTIVE MLP ANALYSIS
📊 Loading VAE output files...
   Data: generated-data-OPTIMIZED.npy
   Windows: generated-data-true-window-OPTIMIZED.npy
✅ Loaded: X=(350000, 650), y=(350000,)
   Data range: [-1.4552, 1.5141]
   Windows range: [2, 24]
   Unique windows: 23

🔗 Analyzing feature-target correlations...
   Max |correlation|: 0.072594
   Mean |correlation|: 0.025965
   Median |correlation|: 0.021110
   Std |correlation|: 0.017960
   Feature breakdown:
     Very weak (<0.01): 117/650 (18.0%)
     Weak (0.01-0.05): 429/650 (66.0%)
     Moderate (0.05-0.1): 104/650 (16.0%)
     Strong (>0.1): 0/650 (0.0%)
   🏗️ Architecture recommendation: WIDE
      (Moderate correlations need width for complexity)

📊 Splitting data...
   Train: 210000 samples, 650 features
   Validation: 70000 samples
   Test: 70000 sa

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')